<style>
.brown {
    color: #9c6644;
    font-weight:700;
    font-size: 40px;
    text-align: button;
    padding: 0;
    margin: 0;
}
</style>
<style>
.light-brown {
    color: #a78a7f;
    font-weight:700;
    font-size: 25px;
    text-align: top;
    padding: 0;
    margin: 0;
}
</style>
<style>
.linea {
  border-top: 2px solid #9c6644;
  height: 2px;
  max-width: 535px;
  padding: 10;
  margin: 10;
}
</style>
<p class="brown">
    PROJECT MLOps STREAMING
</p>

<div class="linea"></div>

<p class="light-brown">
    RECOMMENDATION MODEL
</p>

<style>
.light-brown {
    color: #a78a7f;
    font-weight:700;
    font-size: 20px;
    text-align: top;
    padding: 0;
    margin-top: 40px;
}
</style>
<p class="light-brown">
    LOAD THE DATASET
</p>

In [1]:
# Import the libraries.
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pickle

# Load the dataset.
dfStreamingML = pd.read_parquet('./StreamingML.parquet')
dfStreamingML

,userId,score,id,platform,title,director,cast,country,date_added,release_year,rating,listed_in,duration_int,duration_type
0,1,1.0,as680,amazon prime,the english civil war,no data,robert whelan,no data,NaT,1970,pg-13,"documentary, special interest",1,season
1,583,4.5,as680,amazon prime,the english civil war,no data,robert whelan,no data,NaT,1970,pg-13,"documentary, special interest",1,season
2,765,5.0,as680,amazon prime,the english civil war,no data,robert whelan,no data,NaT,1970,pg-13,"documentary, special interest",1,season
3,2116,3.0,as680,amazon prime,the english civil war,no data,robert whelan,no data,NaT,1970,pg-13,"documentary, special interest",1,season
4,2143,3.0,as680,amazon prime,the english civil war,no data,robert whelan,no data,NaT,1970,pg-13,"documentary, special interest",1,season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024319,123886,3.5,ns720,netflix,power rangers dino fury,no data,"russell curry, hunter deno, kai moya, tessa ra...",united states,2021-06-15,1970,tv-g,kids' tv,1,season
11024320,123923,3.5,ns720,hulu,mlk/fbi,no data,no data,no data,2021-05-14,1970,tv-g,"black stories, documentaries",106,min
11024321,123923,3.5,ns720,netflix,power rangers dino fury,no data,"russell curry, hunter deno, kai moya, tessa ra...",united states,2021-06-15,1970,tv-g,kids' tv,1,season
11024322,124006,3.0,ns720,hulu,mlk/fbi,no data,no data,no data,2021-05-14,1970,tv-g,"black stories, documentaries",106,min


In [2]:
# Define the range of qualifications.
reader = Reader(rating_scale=(1, 5))
# Load the rating data into a Surprise dataset.
data = Dataset.load_from_df(dfStreamingML[['userId', 'title', 'score']], reader)
# Create a training and testing set.
trainSet, testSet = train_test_split(data, test_size=0.25)
# Develop a recommendation model using the SVD algorithm.
model = SVD()
# I train the model, using the training set.
model.fit(trainSet)
# Generate predictions for the test set.
predictions = model.test(testSet)
# Calculate the RMSE and MAE error of the model.
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.9800
MAE:  0.7588
RMSE: 0.9800115468935541
MAE: 0.7587689566984072


In [14]:
# Make a prediction for a specific user and movie.
userId = 124006
id = 'ns720'
prediction = model.predict(userId, id)
# Evaluate the predicted score, to return a corresponding message.
if prediction.est >= 4:
    results = 'Is Totally Recommended!', prediction.est
elif prediction.est >= 3 and prediction.est < 4:
    results = "Is Recommended!", prediction.est
else:
    results = "It might not be to your liking!", prediction.est
# Print the results.
print(f"Your search: {results}")

Your search: ('It might not be to your liking!', 2.462927482913781)


In [ ]:
# Save the trained model to a file using pickle.
with open('modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Create the function to perform the queries.
def recommended(userId, id):
    prediction = model.predict(userId, id)
    if prediction.est >= 4:
        results = 'Is Totally Recommended!', prediction.est
    elif prediction.est >= 3 and prediction.est < 4:
        results = "Is Recommended!", prediction.est
    else:
        results = "It might not be to your liking!", prediction.est
    return results